# Import libraries

In [2]:
import os
import pandas as pd
from argparse import Namespace
from pathlib import Path
import json
import ast

In [3]:
args = Namespace(
    filename= "rantanplan-rhyme-stress-dataset",
    source_dir = Path(os.path.abspath(os.path.join("..", "rantanplan-data", "plain-lyrics"))),
    save_dir = Path(os.path.abspath(os.path.join("..", "rantanplan-data", "rhyme-stress"))),
)

# Helper functions

In [3]:
def get_rhyme(scansion):
    """
    Returns a string containing the rhyme pattern from scansion list of lines.
    """
    if scansion:
        sc = ast.literal_eval(scansion)
        return "".join([line["rhyme"] for line in sc])
    else:
        return None

In [4]:
def get_stress(scansion):
    """
    Returns a string containing the rhyme pattern from scansion list of lines.
    """
    if scansion:
        sc = ast.literal_eval(scansion)
        return [line["rhythm"]['stress'] for line in sc]
    else:
        return None

# Read all files in `source_dir`, extract rhyme and stress and save to a file

In [5]:
%%time
file_list = []
for path in args.source_dir.glob("*.parquet"):
    df = pd.read_parquet(path)
    df['rhyme'] = [get_rhyme(sc) for sc in df['scansion']]
    df['stress'] = [get_stress(sc) for sc in df['scansion']]
    file_list.append(df[["author", "title", "text", "id", "rhyme", "stress"]])
total = pd.concat(file_list)

CPU times: user 1h 42min 13s, sys: 33.9 s, total: 1h 42min 47s
Wall time: 1h 42min 52s


In [16]:
total.sort_values(by=['id'], inplace=True)

In [18]:
total.to_parquet(args.save_dir / f"{args.filename}.parquet",compression="gzip")

In [19]:
total.head(10)

,author,title,text,id,rhyme,stress
0,Los Chunguitos,Dame Veneno,Dame veneno q quiero morir dame veneno\nq ante...,0,abcabc-d-dee--fghfgh-i-ijk-kjlmjlm,"[+--+--+--++--+-, -+--+--+-, --+-+-+--+----+-,..."
1,Los Chunguitos,Cuando La Veo Pasar,"Olee, vamos con ella:\nTodos los días paso por...",1,-a-a-b-bccc-ddeefff-,"[-++--+-, +--++---+-, -+--+-, -+-+-+--+-, --+-..."
2,Alianza,Voy De Frente,Me propuse nadar contra la corriente.\n(Jamás ...,2,abcbacb---bd-edededededededed-,"[--+--+----+-, -+-+--+-, +-+--++--+-, ---+--+-..."
3,Alianza,Ven A Calmar Mi Dolor,¡Hey! ¡Hey!\n¡Hey! ¡Hey!\n¡Hey! ¡Hey!\nSi vien...,3,aaa-a-abbababaabbbaabbbaa-,"[+-, +-, +-, -+--+-, +++--+-, -+-++-, -+-+++-,..."
4,Alianza,Vení A Escuchar Rock & Roll,"Todo ya te pudrió,\nel diario de ayer ya te ca...",4,aaabbabbaaaa-cccccccddddddddedeeddddddddd-,"[+-+--+-, -+--++--+-, -+----+-, -+--+-+++-, -+..."
5,Alianza,Ultimo Round,"Nunca esperes la solución,\ntienes que tomar t...",5,aaaa-aabbbb-cbcbddcbcbcbcb,"[+-+----+-, +---+-+---+-, --+-+--+-, +--++-+--..."
6,Alianza,Tribus Del Sol,"Colonizar y conquistar,\nfue la misión, orden ...",6,aab---b-ccdccdddd,"[---+---+-, +--++-+-, ---++-+---+-, -+-+---+-+..."
7,Alianza,Pasión Prohibida,"No me mires así,\nno es fácil amar sin sufrir....",7,aabbc-cb-bdddee-e-eef-fg-ghhhii-hhhjj-,"[+-+--+-, ++--+--+-, +--+-+-, +-+--+-++-, +--+..."
8,Alianza,Sueño Real,"No importa el tiempo,\nmucho menos el lugar,\n...",8,-ababc-c-ddd-ddd-e-e-f-f-fff-ffffff-fff-,"[++-+-, +-+---+-, ---+-, --+---+-, ---+-, --+-..."
9,Alianza,Sonrisa Fácil,Harto de aguantar\nuna historia más.\nHarto de...,9,aaaaaabb-bccccccccccded-effgeg-egeg-e,"[+---+-, +-+-+-, +---+-, ----+-, -+-+------+-,..."
